In [1]:
#tutorial from https://codeclubprojects.org/en-GB/python/iss/
#https://medium.com/@katehayes.m51/tracking-the-international-space-station-a-mini-project-with-geopandas-e682e8a3489f

In [2]:
import time
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize

In [3]:
response = requests.get("http://api.open-notify.org/iss-now.json")

In [4]:
print(response.status_code)

200


In [5]:
#Originally from the API comes as "byte"
response_content = response.content
print(response_content)
print(type(response_content))

b'{"iss_position": {"longitude": "164.9966", "latitude": "-49.8223"}, "message": "success", "timestamp": 1565201926}'
<class 'bytes'>


In [6]:
#Then json.loads converts it to a dictionary
data = json.loads(response_content)
print(data)
print(type(data))

{'iss_position': {'longitude': '164.9966', 'latitude': '-49.8223'}, 'message': 'success', 'timestamp': 1565201926}
<class 'dict'>


In [7]:
#then json_normalize converts it into a pandas object
df = json_normalize(data)
print(df)
print(type(df))

   message   timestamp iss_position.longitude iss_position.latitude
0  success  1565201926               164.9966              -49.8223
<class 'pandas.core.frame.DataFrame'>


In [8]:
df.head()

,message,timestamp,iss_position.longitude,iss_position.latitude
0,success,1565201926,164.9966,-49.8223


In [9]:
#Create a loop to get ISS info 10 times over every 10 seconds
#response isn't updating and only taking from the first instance
df = pd.DataFrame()
count = 0
while count < 10:
    response = requests.get("http://api.open-notify.org/iss-now.json")
    status = response.status_code
    
    if status != 200:
        print(f"Error improper response code. Code is {status}")
        break
    else:
        response = requests.get("http://api.open-notify.org/iss-now.json")
        resp = response.content
        data_dict = json.loads(resp) # problem is here
        df_temp = json_normalize(data_dict)
        #print(df_temp)
        df = df.append(df_temp)
        time.sleep(10)
        count = count + 1
        print(count)

1
2
3
4
5
6
7
8
9
10


In [10]:
#drops the unneeded message row
df = df.drop(['message'],axis=1)

In [11]:
df.columns = df.columns.str.replace("iss_position.","")

In [12]:
#set the index based on the range of how long the existing index is
df.index = pd.RangeIndex(len(df.index))

In [13]:
print(df.info())
print()
print(df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
timestamp    10 non-null int64
longitude    10 non-null object
latitude     10 non-null object
dtypes: int64(1), object(2)
memory usage: 368.0+ bytes
None

(10, 3)


In [14]:
#need to convert the columns from strings to floats
df['longitude'] = df['longitude'].astype(float)
df['latitude'] = df['latitude'].astype(float)

In [20]:
df

,timestamp,longitude,latitude
0,1565201931,165.4574,-49.7333
1,1565201942,166.4193,-49.5399
2,1565201952,167.3279,-49.3477
3,1565201962,168.2739,-49.1375
4,1565201973,169.2114,-48.9189
5,1565201983,170.0961,-48.7031
6,1565201993,171.0163,-48.4685
7,1565202004,171.9274,-48.2260
8,1565202014,172.7868,-47.9877
9,1565202025,173.6800,-47.7299


In [58]:
timestamp_list = df['timestamp']

In [106]:
date_list = []


for i in range(len(timestamp_list)):
    temp_date_list = str(datetime.date.fromtimestamp(timestamp_list[i]))
    date_list.append(temp_date_list)

In [118]:
time_list = []

for i in range(len(timestamp_list)):
    time_list_datetime = datetime.datetime.fromtimestamp(timestamp_list[i])
    time_list_temp = str(datetime.datetime.time(time_list_datetime))
    time_list.append(time_list_temp)

In [119]:
print(time_list)

['14:18:51', '14:19:02', '14:19:12', '14:19:22', '14:19:33', '14:19:43', '14:19:53', '14:20:04', '14:20:14', '14:20:25']


In [122]:
df['date'] = date_list
df['time'] = time_list
#time list is in miltiary time - hours:minutes:seconds

In [123]:
df

,timestamp,longitude,latitude,datetime,date,time
0,1565201931,165.4574,-49.7333,1970-01-01 00:00:01.565201931,2019-08-07,14:18:51
1,1565201942,166.4193,-49.5399,1970-01-01 00:00:01.565201942,2019-08-07,14:19:02
2,1565201952,167.3279,-49.3477,1970-01-01 00:00:01.565201952,2019-08-07,14:19:12
3,1565201962,168.2739,-49.1375,1970-01-01 00:00:01.565201962,2019-08-07,14:19:22
4,1565201973,169.2114,-48.9189,1970-01-01 00:00:01.565201973,2019-08-07,14:19:33
5,1565201983,170.0961,-48.7031,1970-01-01 00:00:01.565201983,2019-08-07,14:19:43
6,1565201993,171.0163,-48.4685,1970-01-01 00:00:01.565201993,2019-08-07,14:19:53
7,1565202004,171.9274,-48.2260,1970-01-01 00:00:01.565202004,2019-08-07,14:20:04
8,1565202014,172.7868,-47.9877,1970-01-01 00:00:01.565202014,2019-08-07,14:20:14
9,1565202025,173.6800,-47.7299,1970-01-01 00:00:01.565202025,2019-08-07,14:20:25


In [125]:
datetime.datetime.now()

datetime.datetime(2019, 8, 7, 16, 18, 9, 9486)

In [107]:
print(date_list)

['2019-08-07', '2019-08-07', '2019-08-07', '2019-08-07', '2019-08-07', '2019-08-07', '2019-08-07', '2019-08-07', '2019-08-07', '2019-08-07']


In [109]:
print(timestamp_list[0])

1565201931


In [112]:
initial_time = 1565201931
initial_time2 = datetime.datetime.fromtimestamp(initial_time)
time_list = datetime.datetime.time(initial_time2)

In [115]:
print(type(time_list))
print(type(str(time_list)))

<class 'datetime.time'>
<class 'str'>


In [116]:
str(time_list)

'14:18:51'

In [93]:
my_list = ['2','3']

In [94]:
print(my_list)

['2', '3']


In [95]:
my_list.append(temp_list2)

In [96]:
print(type(my_list))
print(type(temp_list2))

<class 'list'>
<class 'str'>


In [99]:
my_list

['2', '3', '2019-08-07']

In [44]:
df['date'] = pd.to_datetime(df['timestamp'])

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
timestamp    10 non-null int64
longitude    10 non-null float64
latitude     10 non-null float64
datetime     10 non-null datetime64[ns]
date         10 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(1)
memory usage: 528.0 bytes


In [29]:
from datetime import datetime

In [38]:
import datetime

In [47]:
conversion = datetime.date.fromtimestamp(timestamp)
time = datetime.time.fromtimestamp(timestamp)

AttributeError: type object 'datetime.time' has no attribute 'fromtimestamp'

In [49]:
conversion2 = datetime.datetime.fromtimestamp(timestamp)

In [64]:
print(conversion2)
print(type(conversion2))

2019-08-07 14:18:51
<class 'datetime.datetime'>


In [57]:
df['new'] = df['timestamp'].apply(datetime.datetime.fromtimestamp(df['timestamp']))

TypeError: cannot convert the series to <class 'int'>

In [52]:
conversion2 = datetime.datetime.fromtimestamp(df['timestamp']).astype(int)

TypeError: cannot convert the series to <class 'int'>

In [40]:
print(conversion)

2019-08-07


In [36]:
timestamp = 1565201931
conversion = datetime.fromtimestamp(timestamp)

AttributeError: module 'datetime' has no attribute 'fromtimestamp'

In [30]:
type(df.timestamp[0])

numpy.int64

In [32]:
df['datetime']=pd.to_datetime(df['timestamp'])

In [33]:
df.head()

,timestamp,longitude,latitude,datetime
0,1565201931,165.4574,-49.7333,1970-01-01 00:00:01.565201931
1,1565201942,166.4193,-49.5399,1970-01-01 00:00:01.565201942
2,1565201952,167.3279,-49.3477,1970-01-01 00:00:01.565201952
3,1565201962,168.2739,-49.1375,1970-01-01 00:00:01.565201962
4,1565201973,169.2114,-48.9189,1970-01-01 00:00:01.565201973


In [ ]:
long_list = df['longitude']
lat_list = df['latitude']

In [ ]:
type(long_list[0])

In [ ]:
for i in range(len(long_list)):
    print(f"The longitude is {long_list[i]} and the latitude is {lat_list[i]}")

In [ ]:
#import turtle
import turtle

In [ ]:
screen = turtle.Screen()
screen.setup(720,360)
screen.bgpic('map.jpg')
screen.setworldcoordinates(-180,-90,180,90)
#screen.register_shape("iss.png")

In [ ]:
iss = turtle.Turtle()
iss.color("red")
iss.pencolor("red")
iss.pensize("5")

In [ ]:
iss.penup()
iss.goto(long_list[0],lat_list[0])

In [ ]:
iss.pendown()

In [ ]:
for i in range(len(long_list)):
    iss.goto(long_list[i],lat_list[i])

In [ ]:
iss.goto(45,45)
iss.goto(50,50)
iss.goto(60,60)
iss.goto(70,70)

In [ ]:
#closes the turtle program
turtle.exitonclick()